In [ ]:
import pandas as pd
import numpy as np

In [1]:
import pandas as pd

file_path = r"C:\Users\Chinelo\Downloads\twitterdata.csv"

try:
    twitter_data = pd.read_csv(file_path, encoding='latin1')  
    print("Data imported successfully!")
    print(twitter_data.head())
except Exception as e:
    print(f"An error occurred: {e}")


Data imported successfully!
   0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY _TheSpecialOne_  \
0  0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   scotthamilton   
1  0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY        mattycus   
2  0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY         ElleCTF   
3  0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY          Karoli   
4  0  1467811372  Mon Apr 06 22:20:00 PDT 2009  NO_QUERY        joy_wolf   

  @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D  
0  is upset that he can't update his Facebook by ...                                                                   
1  @Kenichan I dived many times for the ball. Man...                                                                   
2    my whole body feels itchy and like its on fire                                                                    
3  @nationwideclass no, it's not be

In [2]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
twitter_data

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [4]:
# name the column names and read the dataset again
column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv(file_path, names=column_names, encoding='latin1') 

In [5]:
# check umber of columns and rows
twitter_data.shape

(1600000, 6)

In [6]:
# confirm the column names has been rectify
twitter_data.head(5)

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [7]:
# check for the nnumber of missing values if any
twitter_data.isnull().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

In [8]:
# check the distribution of target values
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [9]:
# convert the target 4 to 1 so 1 is +ve tweet and 0 is -ve tweet.
twitter_data.replace({'target':{4:1}}, inplace= True)

In [10]:
# check the distribution of target values
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [11]:
# seperate the the data and the label.
X = twitter_data['text'].values
Y = twitter_data['target'].values

In [12]:
X

array(["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D",
       "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!",
       '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds',
       ..., 'Are you ready for your MoJo Makeover? Ask me for details ',
       'Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur ',
       'happy #charitytuesday @theNSPCC @SparksCharity @SpeakingUpH4H '],
      shape=(1600000,), dtype=object)

In [13]:
Y

array([0, 0, 0, ..., 1, 1, 1], shape=(1600000,))

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify=Y, random_state = 2)

# X_train and Y_train are linked as Y_train the coresponding label of X.
# X_test and Y_test are linked.

In [15]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [16]:
X_train

array(['about to watch saw iv and drink a lil wine ',
       "@HaterMagazine I'm in! ",
       'even though its my favourite drink i think its the vodka and coke that wipes my mind all the time  think im gonna have to find a new drink',
       ..., 'is eager for Monday afternoon ',
       "Hope everyone and their mother had a great day!  Can't wait to hear what the guys have in store tomorrow!",
       'I love waking up to Folgers. Too bad my voice was deeper than his. '],
      shape=(1280000,), dtype=object)

In [17]:
X_test

array(["@mmangen - M doing fine! I haven't had much time to chat on Twitter.   Hubby is back for the summer &amp; tends to dominate my free time. ",
       'at AHS may show w/  ruth kim &amp; geoffrey sanhueza ',
       '@iShatara maybe it was only a bay area thang  dammit!', ...,
       '@Destini41 Nevertheless Hooray! for 4700 members and have a wonderful and safe trip! ',
       'Not feeling too well ', '@supersandro !!!! thank you! '],
      shape=(320000,), dtype=object)

In [18]:
# The model cant work with text data so we have to convert them to numerical data.
# So, feature extraction is done
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [19]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 15188609 stored elements and shape (1280000, 589439)>
  Coords	Values
  (0, 40489)	0.24120756614816113
  (0, 528767)	0.12112707573781936
  (0, 558927)	0.3035607951325368
  (0, 461482)	0.3283935062719776
  (0, 259193)	0.48840858232447565
  (0, 57473)	0.14918233689170618
  (0, 169798)	0.3832112649811927
  (0, 317443)	0.38662946862459446
  (0, 566317)	0.4123398570437823
  (1, 230652)	0.9765050549311477
  (1, 251985)	0.21549449573925625
  (2, 528767)	0.06899951768639999
  (2, 57473)	0.08498107652774642
  (2, 169798)	0.4365892975560666
  (2, 187743)	0.16659329598173891
  (2, 524174)	0.1594579036569183
  (2, 258127)	0.283734206417887
  (2, 372800)	0.16758245532296176
  (2, 194028)	0.2506594792323131
  (2, 523182)	0.28645427621584585
  (2, 517424)	0.14227030589479395
  (2, 555192)	0.2817247324914906
  (2, 131148)	0.26813732400121504
  (2, 517020)	0.10231390424363242
  (2, 566733)	0.3536332852590365
  :	:
  (1279998, 151216)	0.18667

In [20]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3699189 stored elements and shape (320000, 589439)>
  Coords	Values
  (0, 56126)	0.1488769455924407
  (0, 75487)	0.14090004365001982
  (0, 119150)	0.24248183645807006
  (0, 165538)	0.18104096237371697
  (0, 165924)	0.37505416545454756
  (0, 197441)	0.22153830924032764
  (0, 201909)	0.09906588316358036
  (0, 204417)	0.20529869009944654
  (0, 226659)	0.1462095050939062
  (0, 230930)	0.20171242685035556
  (0, 242946)	0.24412723577494455
  (0, 256825)	0.0961353399433816
  (0, 360633)	0.39258757628101926
  (0, 370285)	0.1543215184822133
  (0, 372800)	0.08816083516534924
  (0, 397223)	0.10689068511219836
  (0, 501895)	0.1910050343105614
  (0, 514780)	0.36028752155055377
  (0, 517424)	0.0748447620172574
  (0, 526358)	0.280616950992062
  (0, 528767)	0.14519551210814186
  (0, 540515)	0.15792536311962385
  (1, 46601)	0.5297958040483101
  (1, 56126)	0.18177926727823518
  (1, 69643)	0.14611121221507706
  :	:
  (319995, 540515)	0.2010976

In [21]:
# logistic regression 
model = LogisticRegression(max_iter=1000)

In [22]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

In [23]:
# accuracy_score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [24]:
print('Accuracy score of the training data:', training_data_accuracy)

Accuracy score of the training data: 0.81171640625


In [26]:
# accuracy_score prediction on the test data using logistics regression
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)


# Accuracy score
print('Accuracy score of the testdata:', test_data_accuracy)

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Precision, Recall, and F1-Score
precision = precision_score(Y_test, X_test_prediction)
recall = recall_score(Y_test, X_test_prediction)
f1 = f1_score(Y_test, X_test_prediction)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')


Accuracy score of the testdata: 0.800215625
Precision: 0.7943597065870832
Recall: 0.8101625
F1-Score: 0.802183283155364
